In [1]:
import cv2
import numpy as np

def detect_teeth(image_path):
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use adaptive thresholding to create a binary image
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area to remove small noise
    min_area = 100  # Adjust this value based on your image
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

    # Draw the contours on the original image
    result = img.copy()
    cv2.drawContours(result, filtered_contours, -1, (0, 255, 0), 2)

    # Display the result
    cv2.imshow('Detected Teeth', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Use the function with your image
detect_teeth('simple.png')


In [7]:
#rough detection WORKING!!!



import cv2
import numpy as np

def detect_landmarks(image_path):
    # Load the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Preprocess: Gaussian blur and adaptive thresholding
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours: Detect tooth contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours: Keep only the largest one (tooth)
    min_area = 100  # Adjust this based on the image
    largest_contour = max([cnt for cnt in contours if cv2.contourArea(cnt) > min_area], key=cv2.contourArea)

    # Approximate Landmarks
    leftmost = tuple(largest_contour[largest_contour[:,:,0].argmin()][0])
    rightmost = tuple(largest_contour[largest_contour[:,:,0].argmax()][0])
    bottommost = tuple(largest_contour[largest_contour[:,:,1].argmax()][0])

    #Estimate landmarks
    cej_approx = ((leftmost[0] + rightmost[0]) // 2, (leftmost[1] + rightmost[1]) // 2)
    root_apex = bottommost

    #Draw the landmarks
    cv2.circle(img, cej_approx, 7, (0, 0, 255), -1) #red
    cv2.circle(img, root_apex, 7, (255, 0, 0), -1) #blue
    cv2.putText(img, "CEJ", (cej_approx[0] - 20, cej_approx[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(img, "Apex", (root_apex[0] - 20, root_apex[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    #Show image for measurement
    cv2.imshow('Landmark Detection', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
detect_landmarks('simple.png')


In [6]:
#CEJ DETECTION DONE FOR ATLEAST 1 IMAGE :)

import cv2
import numpy as np

def getCEJPoint(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Error: Could not open image at {image_path}")
        return
    #Preprocessing
    blurred = cv2.GaussianBlur(img, (7, 7), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    if np.mean(thresh) < 127:
        thresh = cv2.bitwise_not(thresh)

    #Contour time
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("Error: No contours found.")
        return

    #Get the landmarks
    largest_contour = max(contours, key=cv2.contourArea)
    leftmost = tuple(largest_contour[largest_contour[:,:,0].argmin()][0])
    rightmost = tuple(largest_contour[largest_contour[:,:,0].argmax()][0])

    approximateCeJ = ((leftmost[0] + rightmost[0]) // 2, (leftmost[1] + rightmost[1]) // 2)
    #Visualization time

    img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.circle(img_color, approximateCeJ, 5, (0, 0, 255), -1)

    #Putting in some extra help here
    cv2.putText(img_color, "red=CEJ", (approximateCeJ[0] - 20, approximateCeJ[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    #Finishing visualization with the showing the image
    cv2.imshow("Measurements", img_color)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
if __name__ == "__main__":
    image_file = "simple.png"
    getCEJPoint(image_file)


In [3]:
import cv2
import numpy as np

def theLastWhite(image_path):
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Error: Could not open image at {image_path}")
        return

    # Guassian blue to preprocess the image
    blurred = cv2.GaussianBlur(img, (7, 7), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Find "white area"
    if np.mean(thresh) < 127:
        thresh = cv2.bitwise_not(thresh)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("Error: No contours found.")
        return
        
    #Get that image in the contours and return
    largest_contour = max(contours, key=cv2.contourArea)
    bottommost = tuple(largest_contour[largest_contour[:,:,1].argmax()][0])

    img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.circle(img_color, bottommost, 5, (0, 0, 255), -1)
    cv2.putText(img_color, "Bottom Point", (bottommost[0] - 20, bottommost[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Measurements", img_color)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
if __name__ == "__main__":
    image_file = "Picture 50.png"
    theLastWhite(image_file)


In [23]:
#most perfect :)

import cv2
import numpy as np

def detect_landmarks(image_path):
    # Load the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Preprocess: Gaussian blur and adaptive thresholding
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours: Detect tooth contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours: Keep only the largest one (tooth)
    min_area = 100  # Adjust this based on the image
    largest_contour = max([cnt for cnt in contours if cv2.contourArea(cnt) > min_area], key=cv2.contourArea)

    # Approximate Landmarks
    leftmost = tuple(largest_contour[largest_contour[:,:,0].argmin()][0])
    rightmost = tuple(largest_contour[largest_contour[:,:,0].argmax()][0])
    bottommost = tuple(largest_contour[largest_contour[:,:,1].argmax()][0])

    #Estimate landmarks
    cej_approx = ((leftmost[0] + rightmost[0]) // 2, (leftmost[1] + rightmost[1]) // 2)
    root_apex = bottommost

    # Calculate pixel distance
    distance = np.sqrt((cej_approx[0] - root_apex[0])**2 + (cej_approx[1] - root_apex[1])**2)

    #Draw the landmarks
    cv2.circle(img, cej_approx, 7, (0, 0, 255), -1) #red
    cv2.circle(img, root_apex, 7, (255, 0, 0), -1) #blue
    cv2.line(img, cej_approx, root_apex, (0, 255, 0), 2) #green line
    cv2.putText(img, "CEJ", (cej_approx[0] - 20, cej_approx[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(img, "Apex", (root_apex[0] - 20, root_apex[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2.putText(img, f"Distance: {distance:.2f} pixels", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    #Show image for measurement
    cv2.imshow('Landmark Detection', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

detect_landmarks('simple.png')


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

# Load image in grayscale
img_path = "image 7.jpg"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Check if image loaded properly
if img is None:
    print(f"Error: Failed to load image at '{img_path}'. Possible causes:")
    print("- File doesn't exist at specified path")
    print("- File is corrupted")
    print("- Invalid file format")
    exit()

# Verify image dimensions
print(f"Original image size: {img.shape}")

# Resize only if original image is not empty
try:
    img_resized = cv2.resize(img, (256, 512))
except cv2.error as e:
    print(f"Resize failed: {e}")
    exit()

# Continue with processing
height, width = img_resized.shape
gum_line_y = []

for x in range(width):
    column = img_resized[:, x]
    smoothed = gaussian_filter1d(column, sigma=2)
    gradient = np.gradient(smoothed)
    peak_index = np.argmax(gradient)
    gum_line_y.append(peak_index)

gum_line_y = gaussian_filter1d(gum_line_y, sigma=5)

plt.figure(figsize=(10, 6))
plt.title("Extracted Gum Line")
plt.plot(gum_line_y, color='red', linewidth=2)
plt.gca().invert_yaxis()
plt.axis('off')
plt.show()


Error: Failed to load image at 'image 7.jpg'. Possible causes:
- File doesn't exist at specified path
- File is corrupted
- Invalid file format


AttributeError: 'NoneType' object has no attribute 'shape'